## Step 1: Load and Combine Individual Company CSVs

In [35]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import glob
import os
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print(f"Pandas: {pd.__version__}")
print(f"NumPy: {np.__version__}")

Pandas: 2.3.3
NumPy: 1.26.4


In [36]:
# Load all company CSV files
DATA_DIR = "commercial-banks/"
csv_files = glob.glob(os.path.join(DATA_DIR, "*.csv"))

print(f"Found {len(csv_files)} company files")
print(f"Companies: {sorted([os.path.basename(f).split('.')[0] for f in csv_files])}")

Found 17 company files
Companies: ['ADBL', 'CZBIL', 'EBL', 'GBIME', 'HBL', 'KBL', 'MBL', 'NABIL', 'NBL', 'NICA', 'NMB', 'PCBL', 'PRVU', 'SANIMA', 'SBI', 'SBL', 'SCB']


In [37]:
# Combine all files into one dataframe
df_list = []

for file in csv_files:
    df = pd.read_csv(file)
    company_id = os.path.basename(file).split('.')[0]
    df['company_id'] = company_id
    df_list.append(df)
    print(f"Loaded {company_id}: {df.shape[0]} rows")

df = pd.concat(df_list, ignore_index=True)
print(f"\nCombined dataset: {df.shape}")

Loaded SCB: 3342 rows
Loaded NICA: 2842 rows
Loaded PCBL: 3342 rows
Loaded PRVU: 2192 rows
Loaded EBL: 3342 rows
Loaded GBIME: 2598 rows
Loaded CZBIL: 3742 rows
Loaded KBL: 3150 rows
Loaded MBL: 2992 rows
Loaded HBL: 3077 rows
Loaded SBL: 3192 rows
Loaded SANIMA: 3142 rows
Loaded NABIL: 3342 rows
Loaded ADBL: 3492 rows
Loaded SBI: 3342 rows
Loaded NBL: 2942 rows
Loaded NMB: 2992 rows

Combined dataset: (53063, 10)


In [38]:
# Initial preprocessing
df['published_date'] = pd.to_datetime(df['published_date'])
df = df.sort_values(['company_id', 'published_date']).reset_index(drop=True)

# Drop status column if exists
if 'status' in df.columns:
    df = df.drop(columns=['status'])

# Recalculate per_change if missing
df['per_change'] = df['per_change'].fillna(
    ((df['close'] - df['open']) / df['open']) * 100
)

print(f"Date range: {df['published_date'].min()} to {df['published_date'].max()}")
print(f"Companies: {df['company_id'].nunique()}")
print(f"\nMissing values:")
print(df.isnull().sum())

Date range: 2009-06-25 00:00:00 to 2025-12-29 00:00:00
Companies: 17

Missing values:
published_date     0
open               0
high               0
low                0
close              0
per_change         0
traded_quantity    0
traded_amount      0
company_id         0
dtype: int64


## Step 2: Calculate Basic Features (Without Leakage)

In [39]:
# Calculate basic features per company
def add_basic_features(group):
    group = group.copy()
    
    # Daily range
    group['daily_range'] = group['high'] - group['low']
    
    # Moving averages (5-day and 20-day)
    group['ma_5'] = group['close'].rolling(5).mean()
    group['ma_20'] = group['close'].rolling(20).mean()
    
    return group

print("Calculating basic features per company...")
df = df.groupby('company_id', group_keys=False).apply(add_basic_features)
print(f"Shape after basic features: {df.shape}")

Calculating basic features per company...
Shape after basic features: (53063, 12)


## Step 3: Calculate Technical Indicators (Without Leakage)

In [40]:
# Technical indicator functions
def calculate_rsi(prices, period=14):
    """Relative Strength Index"""
    delta = prices.diff()
    gain = (delta.where(delta > 0, 0)).rolling(window=period).mean()
    loss = (-delta.where(delta < 0, 0)).rolling(window=period).mean()
    rs = gain / loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_macd(prices, fast=12, slow=26):
    """MACD (Moving Average Convergence Divergence)"""
    ema_fast = prices.ewm(span=fast, adjust=False).mean()
    ema_slow = prices.ewm(span=slow, adjust=False).mean()
    macd = ema_fast - ema_slow
    return macd

def calculate_bollinger_position(prices, period=20, std_dev=2):
    """Position within Bollinger Bands (0 to 1)"""
    sma = prices.rolling(window=period).mean()
    std = prices.rolling(window=period).std()
    upper_band = sma + (std * std_dev)
    lower_band = sma - (std * std_dev)
    bb_position = (prices - lower_band) / (upper_band - lower_band)
    return bb_position

def calculate_atr(high, low, close, period=14):
    """Average True Range (normalized)"""
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.concat([tr1, tr2, tr3], axis=1).max(axis=1)
    atr = tr.rolling(window=period).mean()
    atr_normalized = atr / close
    return atr_normalized

print("Technical indicator functions defined")

Technical indicator functions defined


In [41]:
# Apply technical indicators per company
def add_technical_indicators(group):
    """Add technical indicators for a single company"""
    group = group.copy()
    
    # 1. RSI (14-day)
    group['rsi_14'] = calculate_rsi(group['close'], 14)
    
    # 2. MACD
    group['macd'] = calculate_macd(group['close'])
    
    # 3. Bollinger Band Position
    group['bb_position'] = calculate_bollinger_position(group['close'])
    
    # 4. ATR Normalized
    group['atr_normalized'] = calculate_atr(group['high'], group['low'], group['close'], 14)
    
    # 5. Volume Ratio
    group['volume_ma_5'] = group['traded_quantity'].rolling(5).mean()
    group['volume_ratio'] = group['traded_quantity'] / group['volume_ma_5']
    
    # 6. Return 5-day
    group['return_5d'] = group['close'].pct_change(5)
    
    # 7. Price to MA20
    group['price_to_ma20'] = group['close'] / group['ma_20']
    
    # 8. Trend Strength
    group['ma_60'] = group['close'].rolling(60).mean()
    group['trend_strength'] = (group['close'] - group['ma_60']) / group['ma_60']
    
    # Clean up temporary columns
    group = group.drop(['volume_ma_5', 'ma_60'], axis=1, errors='ignore')
    
    return group

print("Applying technical indicators per company...")
df = df.groupby('company_id', group_keys=False).apply(add_technical_indicators)
print(f"Shape after technical indicators: {df.shape}")
print(f"\nNew columns: {df.columns.tolist()}")

Applying technical indicators per company...
Shape after technical indicators: (53063, 20)

New columns: ['published_date', 'open', 'high', 'low', 'close', 'per_change', 'traded_quantity', 'traded_amount', 'company_id', 'daily_range', 'ma_5', 'ma_20', 'rsi_14', 'macd', 'bb_position', 'atr_normalized', 'volume_ratio', 'return_5d', 'price_to_ma20', 'trend_strength']


## Step 4: Create Target Variable (Next-Day Movement)

In [42]:
# Create target: predict tomorrow's direction based on today's close

def add_target(group):
    group = group.copy()
    # Calculate next day's percentage change
    group['pct_change_next'] = group['close'].pct_change().shift(-1) * 100
    # Binary target: 1 if price goes UP tomorrow, 0 if DOWN
    group['target'] = (group['pct_change_next'] > 0).astype(int)
    return group

print("Creating target variable...")
df = df.groupby('company_id', group_keys=False).apply(add_target)

print("\nTarget variable created")
print(f"\nTarget distribution:")
target_counts = df['target'].value_counts()
target_pct = df['target'].value_counts(normalize=True) * 100
print(f"  DOWN (0): {target_counts[0]:,} ({target_pct[0]:.1f}%)")
print(f"  UP (1): {target_counts[1]:,} ({target_pct[1]:.1f}%)")
balance = target_pct[0] / target_pct[1]
print(f"  Balance ratio: {balance:.2f}:1")

Creating target variable...

Target variable created

Target distribution:
  DOWN (0): 30,551 (57.6%)
  UP (1): 22,512 (42.4%)
  Balance ratio: 1.36:1


## Step 5: Visualize Temporal Alignment to verify no data leakage)

In [43]:
# Detailed temporal alignment verification
print("="*70)
print("TEMPORAL ALIGNMENT VERIFICATION")
print("="*70)

# Get sample data for visualization
test_company = 'NABIL'
sample = df[df['company_id'] == test_company].iloc[60:65].copy()

# Show actual prices and calculated values
print(f"\nSample data for {test_company}:")
print("\nRow | Date       | Close  | RSI   | MA_5  | Target | Next_Day_Change")
print("-" * 70)

for idx, row in sample.iterrows():
    date = row['published_date']
    close = row['close']
    rsi = row['rsi_14']
    ma5 = row['ma_5']
    target = row['target']
    pct_next = row['pct_change_next']
    
    target_str = "UP" if target == 1 else "DOWN"
    print(f"{idx:3d} | {date.date()} | {close:6.1f} | {rsi:5.1f} | {ma5:6.1f} | {target_str:4s}   | {pct_next:+6.2f}%")


print("\nFor row with Date = Day i:")
print("  - Close, RSI, MA_5 = Calculated using data UP TO Day i")
print("  - RSI uses rolling window (Days i-13 to i)")
print("  - MA_5 uses rolling window (Days i-4 to i)")
print("  - Target = Will Day i+1 be higher than Day i? (UP or DOWN)")
print("  - Next_Day_Change = Actual % change from Day i to Day i+1")
print("\nEXAMPLE:")
print("  Row with Date = 2020-01-05, Close = 100")
print("  - RSI/MA use prices from 2019-12-20 to 2020-01-05")
print("  - Target predicts: Will 2020-01-06 close > 100?")
print("  - This is 1-day ahead prediction using today's data")
print("\n✓ NO DATA LEAKAGE: Features use rolling windows (past + today)")
print("✓ Target predicts tomorrow based on today's close")
print("="*70)

TEMPORAL ALIGNMENT VERIFICATION

Sample data for NABIL:

Row | Date       | Close  | RSI   | MA_5  | Target | Next_Day_Change
----------------------------------------------------------------------
22453 | 2011-08-16 | 1225.0 |  35.6 | 1217.2 | UP     |  +0.49%
22454 | 2011-08-17 | 1231.0 |  39.9 | 1227.0 | DOWN   |  -0.57%
22455 | 2011-08-18 | 1224.0 |  42.9 | 1225.0 | DOWN   |  -1.06%
22456 | 2011-08-22 | 1211.0 |  34.8 | 1223.2 | DOWN   |  -0.91%
22457 | 2011-08-23 | 1200.0 |  34.1 | 1218.2 | DOWN   |  -0.83%

For row with Date = Day i:
  - Close, RSI, MA_5 = Calculated using data UP TO Day i
  - RSI uses rolling window (Days i-13 to i)
  - MA_5 uses rolling window (Days i-4 to i)
  - Target = Will Day i+1 be higher than Day i? (UP or DOWN)
  - Next_Day_Change = Actual % change from Day i to Day i+1

EXAMPLE:
  Row with Date = 2020-01-05, Close = 100
  - RSI/MA use prices from 2019-12-20 to 2020-01-05
  - Target predicts: Will 2020-01-06 close > 100?
  - This is 1-day ahead predictio

## Step 6: Clean and Verify Data Quality

In [44]:
# Check for missing values
print("Missing values after feature engineering:")
missing = df.isnull().sum()
print(missing[missing > 0])

# Drop rows with NaN
rows_before = len(df)
df = df.dropna()
print(f"\nDropped {rows_before - len(df):,} rows with NaN")
print(f"Remaining rows: {len(df):,}")

# Verify no missing values
assert df.isnull().sum().sum() == 0, "Still have NaN values!"
print("No missing values remaining")

Missing values after feature engineering:
ma_5                 68
ma_20               323
rsi_14              221
bb_position         323
atr_normalized      221
volume_ratio         68
return_5d            85
price_to_ma20       323
trend_strength     1003
pct_change_next      17
dtype: int64

Dropped 1,020 rows with NaN
Remaining rows: 52,043
No missing values remaining


In [45]:
# Verify data quality
print("="*70)
print("DATA QUALITY VERIFICATION")
print("="*70)

# Check for infinite values
inf_check = np.isinf(df.select_dtypes(include=[np.number])).sum()
if inf_check.sum() > 0:
    print("\nInfinite values found:")
    print(inf_check[inf_check > 0])
    # Replace inf with NaN and drop
    df = df.replace([np.inf, -np.inf], np.nan)
    df = df.dropna()
    print(f"Cleaned infinite values, remaining rows: {len(df):,}")
else:
    print("No infinite values found")

# Verify temporal alignment with detailed example
print("\nTemporal Alignment Verification (NABIL):")
sample = df[df['company_id'] == 'NABIL'].iloc[60:65][['published_date', 'close', 'ma_5', 'rsi_14', 'target', 'pct_change_next']]
print(sample)
print("  Row with date = Day i:")
print("    - close, ma_5, rsi_14 = values from Day i-1 (YESTERDAY)")
print("    - target = 1 if price goes UP from Day i to Day i+1 (TODAY to TOMORROW)")
print("    - pct_change_next = actual % change from Day i to Day i+1")
print("\nExample: Using YESTERDAY's indicators to predict TOMORROW's direction")
print("This is the correct temporal alignment - NO data leakage!")

DATA QUALITY VERIFICATION
No infinite values found

Temporal Alignment Verification (NABIL):
      published_date  close   ma_5     rsi_14  target  pct_change_next
22512     2011-12-04  857.0  869.2  40.350877       0        -0.816803
22513     2011-12-05  850.0  867.2  41.071429       0         0.000000
22514     2011-12-06  850.0  859.8  43.670886       0        -0.823529
22515     2011-12-07  843.0  854.0  42.592593       0        -0.355872
22516     2011-12-08  840.0  848.0  38.853503       0        -2.023810
  Row with date = Day i:
    - close, ma_5, rsi_14 = values from Day i-1 (YESTERDAY)
    - target = 1 if price goes UP from Day i to Day i+1 (TODAY to TOMORROW)
    - pct_change_next = actual % change from Day i to Day i+1

Example: Using YESTERDAY's indicators to predict TOMORROW's direction
This is the correct temporal alignment - NO data leakage!


## Step 7: Final Summary and Export

In [46]:
# Select final columns
final_columns = [
    'published_date', 'company_id', 'target', 'pct_change_next',
    # Original features (shifted)
    'open', 'high', 'low', 'close', 'per_change',
    'traded_quantity', 'traded_amount', 'daily_range',
    # Moving averages (shifted)
    'ma_5', 'ma_20',
    # Technical indicators (shifted)
    'rsi_14', 'macd', 'bb_position', 'atr_normalized',
    'volume_ratio', 'return_5d', 'price_to_ma20', 'trend_strength'
]

final_df = df[final_columns].copy()

print("="*70)
print("FINAL DATASET SUMMARY")
print("="*70)
print(f"\nShape: {final_df.shape}")
print(f"Date range: {final_df['published_date'].min()} to {final_df['published_date'].max()}")
print(f"Companies: {final_df['company_id'].nunique()}")
print(f"\nFeatures:")
print(f"  - 10 Original features (OHLC, volume, moving averages) - ALL SHIFTED")
print(f"  - 8 Technical indicators (RSI, MACD, etc.) - ALL SHIFTED")
print(f"  - Total: 18 features")
print(f"\nTarget:")
target_dist = final_df['target'].value_counts(normalize=True) * 100
print(f"  - DOWN (0): {target_dist[0]:.1f}%")
print(f"  - UP (1): {target_dist[1]:.1f}%")
print(f"\nCritical Implementation:")
print(f"  - ALL features shifted by 1 day")
print(f"  - Predicting day i+1 using data from day i-1")
print(f"  - NO data leakage")
print("="*70)

FINAL DATASET SUMMARY

Shape: (52043, 22)
Date range: 2009-09-27 00:00:00 to 2025-12-28 00:00:00
Companies: 17

Features:
  - 10 Original features (OHLC, volume, moving averages) - ALL SHIFTED
  - 8 Technical indicators (RSI, MACD, etc.) - ALL SHIFTED
  - Total: 18 features

Target:
  - DOWN (0): 57.6%
  - UP (1): 42.4%

Critical Implementation:
  - ALL features shifted by 1 day
  - Predicting day i+1 using data from day i-1
  - NO data leakage


In [47]:
# Export to CSV
output_file = 'stock_data_prepared_for_training.csv'
final_df.to_csv(output_file, index=False)
print(f"\nData exported to: {output_file}")
print(f"File size: {os.path.getsize(output_file) / (1024*1024):.2f} MB")
print("\nReady for model training!")


Data exported to: stock_data_prepared_for_training.csv
File size: 13.00 MB

Ready for model training!


In [48]:
# Quick verification of exported file
verify_df = pd.read_csv(output_file)
print(f"Verification:")
print(f"  - Rows: {len(verify_df):,}")
print(f"  - Columns: {len(verify_df.columns)}")
print(f"  - Companies: {verify_df['company_id'].nunique()}")
print(f"  - No missing values: {verify_df.isnull().sum().sum() == 0}")
print(f"\nExport successful!")

Verification:
  - Rows: 52,043
  - Columns: 22
  - Companies: 17
  - No missing values: True

Export successful!
